In [2]:
import pandas as pd
from datetime import *

In [4]:
df_b = pd.read_parquet("./data/business.parquet")

In [5]:
df_b.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'working_days', 'features'],
      dtype='object')

In [6]:
df_b.isna().sum()

business_id      0
name             0
address          0
city             0
state            0
postal_code      0
latitude         0
longitude        0
stars            0
review_count     0
is_open          0
working_days    20
features        16
dtype: int64

In [7]:
df_b.shape

(316, 13)